# CNN #
## ** CNN example: ** ##
Adding convolutional layers to MSIST example:
* 1- One CNN layer with 32 filters, 3 * 3 kernel, Relu activation,  28 * 28 pixels
    Stride: default = 1
    Padding: default: no padding ('valid')
    
    Activation map = $ \frac{(D - F + 2P)}{S} + 1 $
    
    
    D: size of image = 28
    F: size of the filter = 3
    P: amount of padding = 0
    S: Stride = 1
    Activation Map = $ ((28 - 3 + 2*0) / 1) + 1) = 26 $
 
    Output: $ 26 * 26 * 32 $

    Params: 1 filter = 3 *3 = 9 weights
            32 filters => 9 * 32 = 288
            32 biases
            Total params = 320

* 2- One CNN layer with 64 filters, 3 * 3 kernel, Relu activation
    Input: 26 * 26 * 32
    Activation Map =  $ \frac{26 - 3 + 2*0}{1} + 1 = 24 $

    Output = 24 *24 * 64 

    Params: 1 filter = 9 weights
    64 filters: 64 * 9 
    each receives 32 from last layer
    Total weights = 64 *9 *32 = 18432
    biases = 64
    Total params: 18496

* 3- Max Pooling Layer: 
    Pool size = 2 * 2
    Stride: default is same as pool size
    Activation Map = 24 / 2 
    (24 size of input divided by pool size)

    Output = 12 * 12 * 64 

* 4- Flatten layer
    Input: 12 *12 * 64
    Flattens to 12 * 12 * 64 = 9216

* 5- One dense layer with 128 units
    Output 128 

    #Params = 9216 * 128 + 128 (Input = 9216, units = 128, biases = 128)
            = 1 179 776

* 6- One output layer, 10 units, Softmax
    Output: 10 probabilties
    #Params = 10 * 128 + 10 = 1290
 

## Results:##
   loss: 0.0207 - accuracy: 0.9944 - val_loss: 0.0623 - val_accuracy: 0.9910

Import packages

In [ ]:
#numpy: Numeric library
import numpy as np 
#graphics
from matplotlib import pyplot as plt

#tensorflow: use some TF tools
import tensorflow as tf
#Keras: Deep learning API. In TF 2.x, it is included in TF
from tensorflow import keras
#Import Dense layer
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
from tensorflow.keras import Model
#Optimizers: SGD (Stochastic Gradien Descent)
#from tensorflow.keras.optimizers import SGD as sgd

#import data set MNIST
from tensorflow.keras.datasets import mnist

#Batch Norm
from tensorflow.keras.layers import BatchNormalization

# import regularizer
from tensorflow.keras import regularizers  

#Dropout
from tensorflow.keras.layers import Dropout

#CNN 
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Activation

Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://localhost:8989/
TypeError: Only absolute URLs are supported

Import MNIST data set
60000 training examples
10000 validation examples
1 example = 28 * 28 matrix (image with 28 * 28 pixels)


In [ ]:
(X_train, y_train), (X_valid, y_valid) = mnist.load_data()


Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://localhost:8989/
TypeError: Only absolute URLs are supported

In [ ]:
print(X_train.shape)


Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://localhost:8989/
TypeError: Only absolute URLs are supported

In [ ]:
print(y_train.shape)

Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://localhost:8989/
TypeError: Only absolute URLs are supported

In [ ]:
print(X_valid.shape)
print(y_valid.shape)

Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://localhost:8989/
TypeError: Only absolute URLs are supported

Process & Prepare data
Reshaping input data:
Input data (x_train[i]) should not be converted to a vector. Is must stay as 60000 of images (28 *28)
- From shape (60000, 28, 28) to shape (60000, 28, 28, 1)
- Last dimension 1 : value of the pixel (monochrome). If it was color image, it would be 3
- Convert to float32 (default is uint8, which python will convert to float64) (flaot 32 will take less memory)

In [ ]:
#use numpy reshape
x_train = X_train.reshape( 60000, 28, 28, 1).astype('float32')
x_valid = X_valid.reshape(10000, 28, 28, 1).astype('float32')

Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://localhost:8989/
TypeError: Only absolute URLs are supported

In [ ]:
print(x_train.shape)
print(x_valid.shape)
print(x_train[0][0][21][0])

Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://localhost:8989/
TypeError: Only absolute URLs are supported

Prepare Data (2)
divide by 255, so the values will range from 0 to 1


Prepare labels (y_train and y_valid)
y_train[i] is given as a number (0, 9). We have 10 possible values. 
Convert to one-hot format:
0 => [1, 0, 0, 0, 0,0, 0,0, 0,0]
1 => [0, 1, 0, 0, 0,0, 0,0, 0,0]
9 => [0, 0, 0, 0, 0,0, 0,0, 0,1]

In [ ]:
n_classes = 10
print(y_train[0])
y_train = keras.utils.to_categorical(y_train, n_classes)
print(y_train[0])

print(y_valid[0])
y_valid = keras.utils.to_categorical(y_valid, n_classes)
print(y_valid[0])


Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://localhost:8989/
TypeError: Only absolute URLs are supported

Define Neural Network Architecture
Using Keras API

In [ ]:
#Define activations
relu = keras.activations.relu
softmax = keras.activations.softmax

#define loss function: Cross entropy
loss = keras.losses.categorical_crossentropy

#define metrics
metrics = [keras.metrics.Accuracy()]

#define optimizer
sgd=keras.optimizers.SGD(lr=0.1)
tf.print(sgd.lr)


Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://localhost:8989/
TypeError: Only absolute URLs are supported

In [ ]:
# Sequential: model of type sequential (layer n can pass information only to layer n+1)
model = Sequential()
model.to_json()

Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://localhost:8989/
TypeError: Only absolute URLs are supported

In [ ]:
#define First Convolutional layer
nbFilters_1 = 32 #Filters for first layer
kernel_size = (3, 3)
input_shape=(28,28,1)

layer1 = Conv2D(nbFilters_1,   kernel_size=kernel_size, activation=relu, input_shape=input_shape)
# No Pooling layer


Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://localhost:8989/
TypeError: Only absolute URLs are supported

In [ ]:
#define Second Convolutional layer
nbFilters_2 = 64 #Filters for first layer
kernel_size = (3, 3)

layer2 = Conv2D(nbFilters_2,   kernel_size=kernel_size, activation=relu)


Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://localhost:8989/
TypeError: Only absolute URLs are supported

In [ ]:
# Add Pooling layer
pool_size=(2,2)
poolingLayer=MaxPool2D(pool_size=pool_size)


Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://localhost:8989/
TypeError: Only absolute URLs are supported

In [ ]:
#Add dense layer 
denseLayer= Dense(128, activation=relu)

#Add outputLayer=
outputLayer = Dense(n_classes, activation=softmax)

Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://localhost:8989/
TypeError: Only absolute URLs are supported

In [ ]:
#Compose Model
model.add(layer1) #Convolutional
model.add(layer2) #Convolutional

model.add(poolingLayer) # Pooling Layer (Max)

model.add(Dropout(0.25)) #Dropout 25%

# Important : Flatten the output so as to add a dense layer
# Dense layer must have a flattened inpit (vector of scalars)
model.add(Flatten())

#Dense layer with dropout
model.add(denseLayer)
model.add(Dropout(0.5))

# Output layer
model.add(outputLayer)


Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://localhost:8989/
TypeError: Only absolute URLs are supported

In [ ]:
#compile Model
model.compile(loss=loss, 
         optimizer='adam', #sgd,
         metrics=['accuracy'])

Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://localhost:8989/
TypeError: Only absolute URLs are supported

In [ ]:
# print model summary
model.summary()

Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://localhost:8989/
TypeError: Only absolute URLs are supported

TensorBoard

In [ ]:
from tensorflow.keras.callbacks import TensorBoard
import os 
import datetime
folder="logs/first_2_cnn_1_dense"
if not os.path.exists(folder):
   os.mkdir(folder)
log_dir = folder + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
print(log_dir)
os.mkdir(log_dir)
 
tensorBoard = TensorBoard(log_dir, histogram_freq=1,profile_batch = 100000000)

Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://localhost:8989/
TypeError: Only absolute URLs are supported

In [ ]:
# Train
batch_size = 128
nb_epoch = 20
history=model.fit(x_train, y_train, batch_size= batch_size, epochs = nb_epoch, verbose=1, validation_data=(x_valid, y_valid),callbacks=[tensorBoard])

Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://localhost:8989/
TypeError: Only absolute URLs are supported

Save Model

In [ ]:
# serialize model to JSON
print(log_dir)
model_json = model.to_json()
with open(log_dir + "/model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save(log_dir + "/model_all.h5")

Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://localhost:8989/
TypeError: Only absolute URLs are supported

Plot History

In [ ]:

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

#Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://localhost:8989/
TypeError: Only absolute URLs are supported

In [ ]:
# summary
model.summary()

Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://localhost:8989/
TypeError: Only absolute URLs are supported

In [ ]:
json = model.to_json()
tf.print(json)

Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://localhost:8989/
TypeError: Only absolute URLs are supported

End